### ¿Qué es un corrector gramatical?

Es un sistema de revisión de textos, capaz de implementar reglas de análisis y corrección de textos basados en el idioma específico con el que se está trabajando.

Capacidad de corrección:

Entre otras cosas, un correcto gramatical puede detectar lo siguiente:

- Faltas de ortografía en palabras comunes  (Por fabor  →  Por favor) 

- Combinación incorrecta de singulares y plurales  (Mi casas  →  Mi casa)

- Palabras repetidas (Le dije eso eso ayer  →  Le dije eso ayer)

- Mayúsculas en lugares incorrectos  (LA tarDe  →  La tarde)

- Problemas con verbos auxiliares (Debes pagado  →  Debes pagar)

Entre otras aplicaciones




In [1499]:
# TAREA 01: Construir un corrector gramatical que revise en un texto errores como los que se detallan a continuación. Imprimir 
# el texto corregido marcando los cambios entre corchetes “[]” Mostrar la cantidad y tipo de correcciones realizadas:

# Tips: 
#- Se debe utilizar un método por cada caso

#- La tarea se trabajará directamente en un Jupyter Notebook y se deberá de poder ejecutar con la instalación realizada en 
# clase, y en caso de usar alguna librería adicional, especificarlo en el mismo notebook


In [1]:
import spacy
from spacy import displacy

nlp = spacy.load("es_core_news_md")

In [2]:
def corrector_ortografico(texto):
    """Esta función verifica la ortografía de las palabras en un texto y corrige errores.

    Args:
        texto: El texto que se va a revisar.

    Returns:
        El texto con los errores corregidos y el número de correcciones.
    """

    # Crear un diccionario de palabras comúnmente mal escritas.
    palabras_incorrectas = {
        "q": "que",
        "x": "por",
        "cmo": "cómo",
        "dnd": "dónde",
        "sq": "es que",
        "sta": "esta",
        "msj": "mensaje",
        "aki": "aquí",
        "tmb": "también",
        "qro": "quiero", 
        "pq": "porque"
    }

    # Crear un documento spaCy a partir del texto.
    doc = nlp(texto)

    texto_corregido = ""
    num_correcciones = 0
    for token in doc:
        if token.text.lower() in palabras_incorrectas:
            texto_corregido += "[" + palabras_incorrectas[token.text.lower()] + "]"
            num_correcciones += 1
        else:
            texto_corregido += token.text

        texto_corregido += " "

    return texto_corregido, num_correcciones

texto = "Hola q tal? Quiero ir aki pq tmb sta el msj dnd me dices sq viste algo x allá."

texto_corregido, num_correcciones = corrector_ortografico(texto)

print("Texto original: {}".format(texto))
print("Texto corregido: {}".format(texto_corregido))
print("Número de correcciones: {}".format(num_correcciones))

Texto original: Hola q tal? Quiero ir aki pq tmb sta el msj dnd me dices sq viste algo x allá.
Texto corregido: Hola [que] tal ? Quiero ir [aquí] [porque] [también] [esta] el [mensaje] [dónde] me dices [es que] viste algo [por] allá . 
Número de correcciones: 9


In [1502]:
# Caso 2: Verificar que no exista una palabra en singular, seguida de una en plural y al revés

In [52]:
nlp = spacy.load("es_core_news_md")

text = "Los coche son rojos. Mi amigos es amable."
doc = nlp(text)

correcciones = []

def correct_number_mismatches(doc):    
    for i, token in enumerate(doc[:-1]):  # Usamos doc[:-1] para evitar acceder fuera de rango
        next_tok = doc[i + 1]

        
        # Corregimos las comparaciones
        if token.tag_ == 'DET' and token.text[-1] == 's' and next_tok.tag_ == 'NOUN' and next_tok.text[-1] != 's':
            correcciones.append((next_tok.text, next_tok.text + 's'))
        elif token.tag_=='DET' and token.text[-1] != 's' and next_tok.tag_ == 'NOUN' and next_tok.text[-1] == 's':
            correcciones.append((next_tok.text, next_tok.text[:-1]))

def apply_corrections(text, correcciones):
    for original, corrected in correcciones:
        text = text.replace(original, corrected)
    return text

correct_number_mismatches(doc)

texto_corregido = apply_corrections(text, correcciones)
print("Texto original:", text)
print('Correcciones sugeridas: ', correcciones)
print("Texto corregido:", texto_corregido)



Correcciones sugeridas:  [('coche', 'coches'), ('amigos', 'amigo')]
Texto original: Los coche son rojos. Mi amigos es amable.
Texto corregido: Los coches son rojos. Mi amigo es amable.


In [7]:
# Caso 3: Verificar que no exista la misma palabra dos veces seguidas

In [53]:
def no_repetir_palabras(texto):
    """Esta función verifica que no exista la misma palabra repetida dos veces seguidas en un texto.

    Args:
        texto: El texto a revisar.

    Returns:
        El texto con las correcciones y el número de correcciones.
    """

    # Crear un documento spaCy a partir del texto.
    doc = nlp(texto)

    texto_corregido = ""
    num_correcciones = 0

    prev_token = None
    for token in doc:
        if prev_token is not None and token.text.lower() == prev_token.text.lower():
            num_correcciones += 1
            continue
        else:
            texto_corregido += token.text + " "
            prev_token = token

    return texto_corregido.strip(), num_correcciones

texto = "Estamos probando probando que no haya palabras palabras repetidas en este este texto"

texto_corregido, num_correcciones = no_repetir_palabras(texto)

print("Texto original: {}".format(texto))
print("Texto corregido: {}".format(texto_corregido))
print("Número de correcciones: {}".format(num_correcciones))

Texto original: Estamos probando probando que no haya palabras palabras repetidas en este este texto
Texto corregido: Estamos probando que no haya palabras repetidas en este texto
Número de correcciones: 3


In [1505]:
# Caso 4: Verificar que la forma de un token no mezcle mayúsculas, minúsculas y dígitos, a menos que se trate de todo en mayúsculas para marcar siglas 

In [ ]:
# FIXME solo mayúsculas en el incio, ya te dice si si están combinadas o no

In [62]:
import spacy

nlp = spacy.load("es_core_news_sm")

def check_token_case(token):
    """Verifica si el token cumple con la regla de mayúsculas y minúsculas.

    Args:
        token (Token): El token a verificar.

    Returns:
        bool: True si el token cumple con la regla, False si no.
    """
    text = token.text

    if text.isupper():  # Todo en mayúsculas (siglas están permitidas)
        return True
    elif text.islower():  # Todo en minúsculas
        return True
    elif text.isdigit():  # Solo dígitos
        return True
    else:
        # Verificar que no tenga combinación de mayúsculas, minúsculas y dígitos
        if any(char.islower() for char in text) and any(char.isupper() for char in text) or any(char.isdigit() for char in text):
            return False
    return True  # Cualquier otra combinación no especificada se considera válida

def check_token_case_rule(text):
    doc = nlp(text)
    for token in doc:
        if not check_token_case(token):
            return False
    return True

# Ejemplo de uso para verificar regla de mayúsculas y minúsculas
texto_mayus = "los gatos son animales"
if not check_token_case_rule(texto_mayus):
    print("El texto combina mayúsculas y minúsculas de manera incorrecta.")
else:
    print("El texto no combina mayúsculas y minúsculas de manera incorrecta.")


El texto no combina mayúsculas y minúsculas de manera incorrecta.


In [1507]:
# Caso 5: Identificar cuando se escriben 2 “NOUNS” seguidos, pero no se trata de un NOMBRE propio

In [1508]:
def detectar_dos_nouns_seguidos(texto):
    """Esta función identifica pares de "NOUNS" seguidos que no son nombres propios y los corrige.

    Args:
        texto: El texto a revisar.

    Returns:
        Una lista de segundos sustantivos y el texto corregido sin los segundos sustantivos.
    """

    # Crear un documento spaCy a partir del texto.
    doc = nlp(texto)

    segundos_sustantivos = []
    texto_corregido = ""

    token_ant = None
    for token in doc:
        if token_ant is not None and token_ant.pos_ == "NOUN" and token.pos_ == "NOUN" and token.ent_type_ != "PERSON":
            if token_ant.text.lower() != token.text.lower():
                segundos_sustantivos.append(token.text)
                if token_ant.text in texto_corregido and token.text in texto_corregido:
                    texto_corregido = texto_corregido.replace(token_ant.text + " " + token.text, "[" + token_ant.text + " " + token.text + "]", 1)
            else:
                texto_corregido += " " + token.text
        else:
            texto_corregido += " " + token.text
        token_ant = token

    return segundos_sustantivos, texto_corregido.strip()

texto = "Este año ciudad, nos vamos a mudar a la ciudad año!"

segundos_sustantivos, texto_corregido = detectar_dos_nouns_seguidos(texto)

print("Segundos sustantivos encontrados:", segundos_sustantivos)
print("Texto original: {}".format(texto))
print("Texto corregido: {}".format(texto_corregido))


Segundos sustantivos encontrados: ['ciudad', 'año']
Texto original: Este año ciudad, nos vamos a mudar a la ciudad año!
Texto corregido: Este año , nos vamos a mudar a la ciudad !
